In [1]:
import openai
import json
from sklearn.metrics import classification_report
import pandas as pd
from tqdm import tqdm
from credentials import OPENAPI_KEY
openai.api_key = OPENAPI_KEY

In [ ]:
train_ds_path = "DS_PATH/train.jsonl"
openai.File.create(file=open(train_ds_path, "rb"), purpose="fine-tune")

In [ ]:
openai.FineTuningJob.create(training_file="FILE_ID", model="gpt-3.5-turbo")

In [ ]:
openai.FineTuningJob.retrieve("FINETUNE_ID")

In [113]:
test_ds_path = "DS_PATH/test.jsonl"
with open(test_ds_path, "r") as f:
    test_dataset = [json.loads(line) for line in f]

In [119]:
test_dataset[0]["messages"]

[{'role': 'system',
  'content': 'vrl-gpt3.5-turbo is a chatbot that can give the sentiment of Turkish texts.'},
 {'role': 'user',
  'content': 'What is the sentiment of this Turkish text "@carvalhogh_ FELIZ 2021 kdkakdsk"?'},
 {'role': 'assistant', 'content': 'neutral'}]

In [121]:
preds = []
refs = []
for example in tqdm(test_dataset):
    completion = openai.ChatCompletion.create(
        model="MODEL_ID",
        messages=example["messages"][:2]
        )
    model_output = completion.choices[0].message
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output["content"])
    refs.append(ground_truth)

100%|██████████| 23689/23689 [2:04:56<00:00,  3.16it/s]  


In [122]:
output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv("OUTPUT_PATH/output.csv", index=False)

In [133]:
output_labels = ["negative", "neutral", "positive"]
for index in range(len(preds)):
    if preds[index] not in output_labels:
        preds[index] = "neutral" 

In [140]:
print(classification_report(refs, preds, digits=4))

              precision    recall  f1-score   support

    negative     0.5706    0.6154    0.5922      8074
     neutral     0.5552    0.5421    0.5485     10146
    positive     0.5323    0.4939    0.5124      5469

    accuracy                         0.5560     23689
   macro avg     0.5527    0.5505    0.5510     23689
weighted avg     0.5552    0.5560    0.5551     23689



In [136]:
temp_df  = pd.read_csv("OUTPUT_PATH/output.csv")

temp_df.head()

,preds,refs
0,positive,neutral
1,neutral,neutral
2,neutral,neutral
3,neutral,neutral
4,neutral,negative


In [137]:
temp_df["preds"].value_counts()

preds
neutral               9890
negative              8708
positive              5074
netural                  1
unknown                  1
pos                      1
neagtive                 1
joyful                   1
Positive                 1
Neutral                  1
sadness                  1
neutrally_positive       1
 negative                1
mixed                    1
surprise                 1
😂 positive               1
other                    1
joy                      1
nefret                   1
neut                     1
Name: count, dtype: int64